In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import numpy as np
from tqdm import trange

#
from visualize import Visualize
from track import Track

#
import cv2

Autosaving every 180 seconds


2022-12-08 11:11:35.599312: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cat/miniconda3/envs/gerbils/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-08 11:11:35.599346: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

def extract_movie_frame(fname_movie,
                        frame_number):
    
    import cv2
    cap = cv2.VideoCapture(fname_movie)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    res, frame0 = cap.read()
    res, frame1 = cap.read()

    return frame0, frame1

########################################
########################################
########################################
def detect_id_switch(fname_movie,
                     tracks, 
                     animal_id,
                     min_n_features=4):
    
    #
    other_animal_ids = np.int32(np.delete(np.arange(6), animal_id))
   # print (other_animal_ids)
    
    
    # check for continuity in a track
    track = tracks[:,animal_id]
    #print (track.shape)
    
    # calculate the feature-wise distance between current frame and next frame
    ctr=0
    for k in trange(tracks.shape[0]-1):
        feats0 = track[k]
        
        for a in other_animal_ids:
            feats1 = tracks[k+1,a]
            
            #
            dists = np.linalg.norm(feats1-feats0,axis=1)
            #print (dists)

            # 
            sums = np.nanmedian(dists)
            #print (sums)
            
            if sums<10:
                n = np.count_nonzero(np.isnan(dists))
                if n<=(6-min_n_features):
                    #print (n)
                    #print (dists)
                    #print (sums)
                    frame0, frame1 = extract_movie_frame(fname_movie,
                                                k)
                    #if ctr<5:
                    plot_flip2(k,
                                   frame0,
                                   frame1,
                                   tracks[k],
                                   tracks[k+1]
                                  )

                    ctr+=1
                #break
                
        #if sums<10:
        #    break

def detect_id_switch_no_plot(fname_movie,
                             tracks, 
                             animal_id,
                             min_n_features=4):

    #
    other_animal_ids = np.int32(np.delete(np.arange(6), animal_id))
    
    # select animal_id track
    track = tracks[:,animal_id]
    
    # calculate the feature-wise distance between current frame and next frame
    ctr=0
    frame_ids = []
    corner=300
    for k in trange(tracks.shape[0]-1, desc='finding id switches in movie'):
        feats0 = track[k]
        
        for a in other_animal_ids:
            feats1 = tracks[k+1,a]
            
            #
            dists = np.linalg.norm(feats1-feats0,axis=1)

            # 
            sums = np.nanmedian(dists)
            
            if sums<10:
                #
                n = np.count_nonzero(np.isnan(dists))
                if n<=(6-min_n_features):
                    # also skip switches in the huddle
                    if np.nanmedian(feats0[:,0])>corner:
                        if np.nanmedian(feats0[:,1])>corner:

                            frame_ids.append([k,k+1])

                        ctr+=1
    
    return frame_ids
        
########################################
########################################
########################################

def plot_flip2(n,
               frame0,
               frame1,
               track0,
              track1):
    
    clrs = ['blue','red','pink','cyan','magenta','lightgreen',
    ]
    
    
    #print ("track0: ", track0)
    
    plt.figure(figsize=(15,15))
    
    ax=plt.subplot(121)
    ax.imshow(frame0)
    
    for k in range(6):
        feats0 = track0[k]
        #print("Feats0: ", feats0, feats0.shape)
        if len(feats0.shape)>1:
    
            plt.scatter(feats0[:,0],
                   feats0[:,1],
                   c=clrs[k])
    
    #
    ax=plt.subplot(122)
    ax.imshow(frame1)
    for k in range(6):
        feats1 = track1[k]
        #print("Feats1: ", feats1)
        
        if len(feats1.shape)>1:
            plt.scatter(feats1[:,0],
                    feats1[:,1],
                    c=clrs[k])
    
    plt.savefig('/home/cat/example_'+str(n)+'.png')
    plt.close()
    
########################################
########################################
########################################

def plot_flip(frame0,
              frame1,
              animal_id,
              feats0,
              feats1,
             a):
    
    clrs = ['blue','red','pink','cyan','magenta','lightgreen',
    
    
    ]
    
    plt.figure()
    
    ax=plt.subplot(121)
    ax.imshow(frame0)
    plt.scatter(feats0[:,0],
               feats0[:,1],
               c=clrs[animal_id])
    
    #
    ax=plt.subplot(122)
    ax.imshow(frame1)
    plt.scatter(feats1[:,0],
               feats1[:,1],
               c=clrs[a])
    

    plt.show()
    

In [3]:
######################################################## 
################ GENERATE FIXED TRACK ##################
########################################################

fnames = np.loadtxt('/media/cat/256GB/dan/id_switch/fnames.txt',dtype='str')

for fname_slp in fnames:
    # 
    #fname_slp = '/home/cat/data/dan/2020_07_30_00_34_28_393297/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
    track = Track(fname_slp)
    track.track_type = 'features'
    track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
    track.load_tracks()

    print (track.tracks.shape)


track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
(28802, 6, 6, 2)
track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_11_06_03_274369_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
(28802, 6, 6, 2)
track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_14_11_50_567584_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
(28802, 6, 6, 2)
track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_17_34_43_625922_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), t

In [26]:
########################################
########################################
########################################

fname_movie = '/home/cat/data/dan/2020_07_30_00_34_28_393297/2020_07_30_00_34_28_393297_compressed_defished_shrink_cropped.mp4'
animal_id = 0
detect_id_switch(fname_movie,
                 track.tracks, 
                 animal_id)

  2%|▊                                                | 474/28801 [00:00<00:06, 4150.37it/s]


TypeError: Image data of dtype object cannot be converted to float

In [15]:
#
def plot_feats(feats,
           colors,
           img_out,
           dot_size):

    for a in range(feats.shape[0]):
        color = colors[a]  # (255, 0, 0)
        #print ("animal id: ", a, ", color: ", color)
        for f in range(feats.shape[1]):
            x = feats[a, f, 0]
            y = feats[a, f, 1]

            if np.isnan(x) or np.isnan(y):
                continue

            x = int(x)
            y = int(y)

            # plot cicrl
            center_coordinates = (x, y)
            img_out = cv2.circle(img_out, 
                                 center_coordinates, 
                                 dot_size, 
                                 color, 
                                 -1)
   # print ('')
    return img_out


import glob
fnames = np.loadtxt('/home/cat/data/dan/id_switch/fnames_day.txt',dtype='str')


#
colors = [
    (0, 0, 255),
    (255, 0, 0),
    (255, 255, 0),
    (0, 128, 0),
    (128,128, 128),
    (128, 0, 128),
]


#
animal_ids = np.arange(6)

# load and videos

fname_out = '/home/cat/data/dan/id_switch/movie_hybrid.mp4' 
fname_out_annotated = fname_out[:-4]+"_annotated.mp4"

# make new video video settings
size_vid = np.int32(np.array([900, 700]))
fps_out=1
dot_size = int(6)
thickness = -1
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#fourcc = cv2.VideoWriter_fourcc(*'X264')
#fourcc = cv2.VideoWriter_fourcc(*args["codec"])
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
#
video_out = cv2.VideoWriter(fname_out, 
                           fourcc, 
                           fps_out, 
                           (size_vid[0], size_vid[1]), 
                           True)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video_out = cv2.VideoWriter(fname_out,
                            fourcc, 
                            20.0, (900,700))


print ("Saving video: ", fname_out)
#
# video_out_annotated = cv2.VideoWriter(fname_out_annotated, 
#                                       fourcc, 
#                                       fps_out, 
#                                       (size_vid[0], size_vid[1]), 
#                                       True)
# #
for fname_slp in fnames:
    print ("Loading: ", fname_slp)
    
    #fname_slp = '/home/cat/data/dan/2020_07_30_00_34_28_393297/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
    track = Track(fname_slp)
    track.track_type = 'features'
    track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
    track.load_tracks()
    
    #
    fname_movie = glob.glob(fname_slp[:-10]+"*.mp4")[0]
    print ("movie: ", fname_movie)
    
    #### FIND ALL ID SWITCHES IN MOVIE ####
    all_frames = []
    
    #for animal_id in animal_ids:
    for animal_id in [2]:
        frame_ids = detect_id_switch_no_plot(fname_movie,
                                             track.tracks, 
                                             animal_id)
        if len(frame_ids)>0:
            all_frames.append(frame_ids)
    
    #
    all_frames = np.vstack(all_frames)
    print ("all frames: ", all_frames.shape)
    
    # load current vid
    original_vid = cv2.VideoCapture(fname_movie)
    width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
    height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )
    
    # set frames to new ones
    ctr=0
    for frames in all_frames:
        print ("frame #s: ", frames)
        if len(frames)==0:
            continue

        #
        start = 0
        original_vid.set(cv2.CAP_PROP_POS_FRAMES, frames[0]+start)
        for z in range(2):
            
            ret, img_out = original_vid.read()
            #plt.figure()
            #plt.imshow(img_out)
            #plt.show()
        
            # save non-annotated video
            video_out.write(img_out)
            
            #
#            if False:
#             feats = track.tracks[frames[0]+start+z]

#             #            
#             img_out = plot_feats(
#                                 feats,
#                                 colors,
#                                 img_out,
#                                 dot_size)
            
            
#             video_out_annotated.write(img_out)
#         video_out_annotated.write(img_out*0)

    break
    
video_out.release()
#video_out_annotated.release()
print ("DONE...")

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Saving video:  /home/cat/data/dan/id_switch/movie_hybrid.mp4
Loading:  /home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.slp
movie:  /home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4


finding id switches in movie: 100%|█████| 28801/28801 [00:04<00:00, 5769.74it/s]


all frames:  (8, 2)
frame #s:  [12959 12960]
frame #s:  [13016 13017]
frame #s:  [14566 14567]
frame #s:  [14678 14679]
frame #s:  [15296 15297]
frame #s:  [15304 15305]
frame #s:  [16416 16417]
frame #s:  [16580 16581]
DONE...


In [ ]:
import av
import av.datasets
codec = av.CodecContext.create("h264", "r")

fname = '/home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4'
input_ = av.open(av.datasets.curated(fname))
output = av.open(fname[:-4]+"_remuxed.mkv", "w")

# Make an output stream using the input as a template. This copies the stream
# setup from one to the other.
in_stream = input_.streams.video[0]
out_stream = output.add_stream(template=in_stream)


ctr=0
from tqdm import tqdm
for packet in tqdm(input_.demux(in_stream)):

    #print(packet)

    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        print ("ERROR")
        continue

    # We need to assign the packet to the new stream.
    packet.stream = out_stream

    #frames = codec.decode(packet)
    print ("packet", packet.stream)
    print ('frames;', frames)
        
    output.mux(packet)
    
    ctr+=1
    if ctr>5000:
        break
input_.close()
output.close()





0it [00:00, ?it/s]

In [1]:


import av
fname = '/home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4'

v = av.open(fname)

ctr=0
for packet in container.demux():
    for frame in packet.decode():
        if frame.type == 'video':
            img = frame.to_image()  # PIL/Pillow image
            arr = np.asarray(img)  # numpy array
            # Do something!

    ctr+=1
    
    if ctr>500:
        break
            
            

NameError: name 'container' is not defined

In [8]:
import imageio as iio
import numpy as np


index = [1,2,3,4,5,6,7,8]

with iio.imopen("imageio:/home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4"+fname, "r", plugin="pyav") as img_file:
    img_list = [img_file.read(index=idx) for idx in index]

img_array = np.stack(img_list)




ValueError: Unknown standard image '/home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4'.

In [5]:
import imageio
#filename = '/tmp/file.mp4'
fname = '/home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4'


vid = imageio.get_reader(fname,  'ffmpeg')

for image in vid.iter_data():
    print(image.mean())

metadata = vid.get_meta_data()

86.70925954182943
80.1605613708496
79.94119262695312
83.58482538859049
86.24884948730468
85.04408594767253
80.45683975219727
79.97509435017903
83.20458730061848
85.99119262695312
84.05765431722006
80.6570826212565
78.91300735473632
83.42650477091472
86.89409790039062
84.66044158935547
79.9940907796224
79.58783187866212
83.40449752807618
86.24143600463867
85.28732223510742
80.68789342244466
80.09318567911784
83.18669179280599
85.65164591471354
85.20053685506186
80.32417831420898
79.17085342407226
82.8681241353353
86.28439127604166
86.09142812093098
80.6971066792806
79.42621688842773
82.57780380249024
86.19281336466472
85.77793197631836
81.45068613688152
80.30224278767903
82.95350723266601
86.18602574666342
85.73253682454427
81.64940439860025
80.03501561482747
82.8228858947754
85.95749003092448
85.76552734375
81.56600570678711
79.76457163492839
82.05664978027343
85.99529088338217
85.9548568725586
81.6289555867513
79.87899169921874
81.83612060546875
85.56517130533854
85.63741302490234
81.

KeyboardInterrupt: 

In [14]:



import glob
fnames = np.loadtxt('/home/cat/data/dan/id_switch/fnames_day.txt',dtype='str')


#
colors = [
    (0, 0, 255),
    (255, 0, 0),
    (255, 255, 0),
    (0, 128, 0),
    (128,128, 128),
    (128, 0, 128),
]


#
animal_ids = np.arange(6)

# load and videos

fname_out = '/home/cat/data/dan/id_switch/movie_hybrid.mp4' 
fname_out_annotated = fname_out[:-4]+"_annotated.mp4"

# make new video video settings
size_vid = np.int32(np.array([900, 700]))
fps_out=10
dot_size = int(6)
thickness = -1
#fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#fourcc = cv2.VideoWriter_fourcc(*'X264')
#fourcc = cv2.VideoWriter_fourcc(*args["codec"])
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
#
#video_out = cv2.VideoWriter(fname_out, 
#                           fourcc, 
#                           fps_out, 
#                           (size_vid[0], size_vid[1]), 
#                           True)

#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video_out = cv2.VideoWriter(fname_out,
                            fourcc, 
                            20.0, (900,700))


print ("Saving video: ", fname_out)
#
# video_out_annotated = cv2.VideoWriter(fname_out_annotated, 
#                                       fourcc, 
#                                       fps_out, 
#                                       (size_vid[0], size_vid[1]), 
#                                       True)
# #
for fname_slp in fnames:
    print ("Loading: ", fname_slp)
    
    #fname_slp = '/home/cat/data/dan/2020_07_30_00_34_28_393297/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
    track = Track(fname_slp)
    track.track_type = 'features'
    track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
    track.load_tracks()
    
    #
    fname_movie = glob.glob(fname_slp[:-10]+"*.mp4")[0]
    print ("movie: ", fname_movie)
        
    # load current vid
    original_vid = cv2.VideoCapture(fname_movie)
    width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
    height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )
    
    # set frames to new ones
    ctr=0
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, 1)
    for k in trange(1000):

        ret, img_out = original_vid.read()
        
        # save non-annotated video
        video_out.write(img_out)
        
    break
    
video_out.release()
#video_out_annotated.release()
print ("DONE...")

Saving video:  /home/cat/data/dan/id_switch/movie_hybrid.mp4
Loading:  /home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.slp
movie:  /home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4


100%|██████████████████████████████████████| 1000/1000 [00:02<00:00, 352.24it/s]

DONE...


In [57]:
#
print ("all frames: ", all_frames.shape)
clrs = ['black','blue','red','magenta','cyan','green']
# load current vid
original_vid = cv2.VideoCapture(fname_movie)
width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )

# set frames to new ones
frames = all_frames[0]
    
print ("frame #s: ", frames)

#
plt.figure()
#
for z in range(2):
    ax=plt.subplot(1,2,z+1)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, 
                     frames[z])

    ret, img_out = original_vid.read()

    # save non-annotated video
    video_out.write(img_out)

    #
    feats = track.tracks[frames[z]]
    for k in range(6):
        plt.scatter(feats[k,:,0],
                    feats[k,:,1],
                   c=clrs[k])

    plt.ylim(700,0)
plt.show()

all frames:  (8, 2)
frame #s:  [15000 15001]


In [10]:
####################################################
########## MAKE PLOTS OF SPINE CENTRES #############
####################################################
plt.figure()
for k in range(0,6,1):
    #
    #ax=plt.subplot(2,3,k+1)
    plt.plot(track.tracks_spine[:,k,0],
             track.tracks_spine[:,k,1],
             label='spine2',
             
            )
    #
    # #print (track.tracks.shape)
    # plt.plot(np.nanmedian(track.tracks[:,k,:,0]+2,axis=1),
    #          np.nanmedian(track.tracks[:,k,:,1]+2,axis=1),
    #          label='raw (median of all body parts)',
    #          alpha=.5
    #         )
   
    if k==1:
        plt.legend()
    plt.xlim(0,1000)
    plt.ylim(0,800)
    #break
plt.show()

In [13]:
####################################################
########## MAKE MOVIES OF SPINE CENTRES ############
####################################################
print ("")
print ("Making movie...")

#
vis = Visualize()
fps = 25
start = 0*fps       # frame to start in seconds
end = 60*fps        # frame to end in seconds
fps = 25

#
fname_video = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed.mp4'
fname_video_extension = '_fixed'  # the extension to be added to the filename

# Note; thre is also a version of this that plots all body parts
# - this function takes in only a single animal location 
shrink = 1 # shrink image factor

fname_track = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed_Day_huddle_multi_track_huddles.npy'
track = np.load(fname_track)
print ("track shape: ", track.shape)
#print (track.tracks_spine.shape)
#
vis.make_video_centroid(track,                # visualize centres locations
                        fname_video,
                        fname_video_extension,
                        start,
                        end,
                        fps,
                        shrink) 

# 
print ("DONE...")

  0%|          | 5/1500 [00:00<00:32, 46.07it/s]


Making movie...
track shape:  (28802, 10, 2)
width, heigh:  1280.0 1024.0
Histories:  (10, 5, 2)


100%|██████████| 1500/1500 [00:28<00:00, 52.92it/s]

DONE...
